In [1]:
#Job Parameters
job_id=14
job_name= 'PackageCount_Volume_at_Facility'

# #Snowflake Input Credentials
user_id='FUSION_FDR_ETL_USER'
passcode='FsW@1fDrR'
WAREHOUSE='FUSION_CS_DWH'
snowflake_account='pitneybowes.us-east-1'

#Snowflake Output Parameters
snowflake_output_database = 'FUSION_FDR_DB'
snowflake_output_schema = 'FUSION_ANOMOLY_DEV_SCHEMA'
snowflake_output_table = 'ANOMALY_BACKTESTING_TIMESERIES'

In [2]:
#import packages
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import snowflake.connector
import itertools
from prophet import Prophet
import matplotlib.pyplot as plt
import time
from tqdm import *
import warnings

Importing plotly failed. Interactive plots will not work.


In [3]:
#Training Parameters 
#total days
test_last_days = 90

# length of training period (days)
training_period=test_last_days-60

# training interval increment (days)
interval=1

# window between checkpoint (hours)
window='24H'

#number of forecast (based on windows between checkpoint)
number_of_forecast = 24/int(window[0])

#group-by variable for printing during training
category_1='Facility'
category_2='Event'

#list of confidence interval
confidence_interval=[0.80,0.90,0.95,0.99]   

# Format output file
monitor_name = 'PackageCount_Volume_at_Facility'
lob = 'Tracking'
version = 1.0

In [4]:
#start and end date filter for data query
et=datetime.today().date() - timedelta(days = 1)
st=et-timedelta(days=test_last_days)

#query from Snowflake
query = '''
            SELECT CAST(EVENTRPTDATETIME as DATE) AS Event_Time,year(CAST(EVENTRPTDATETIME as DATE)) year,
                    FWDTRACKINGID AS Tracking_ID, 
                    EVENTFACILITYNAME as Facility, 
                    TRACKINGEVENTKEY as Event
            FROM "FDR_DWH_DB_PROD"."DELIVERY"."FWDTRACKINGEVENT"
            WHERE eventrptdatetime <= '''+"'"+str(et)+" 00:00:00'"+'''
            AND eventrptdatetime >= '''+"'"+str(st)+" 00:00:00'"+'''
            AND EVENTFACILITYNAME IN ('IND3','EWR2','ATL4','ONT1','RNO2','DFW5') AND
            TRACKINGEVENTKEY in (7,14,335) 
           and year='2021'
            ;
        '''
#             and eventfacilityname in ('IND3','EWR2','ATL4','ONT1','RNO2','DFW4','CVG1','ORD2','BWI1','MCO1') 
#             and trackingeventkey in (7,14,33,335,334,326) 

In [5]:
#import from functions.py
from functions import createConnection,extractData,exploratory_data_analysis,preprocessing

In [6]:
#connect to Snowflake and query data
createConnection(user_id,passcode)
df=extractData(st,et,query)

Attempting to establish a Secure Connection...
Connected Successfully.
Data Extraction Started
Data Extraction Completed


In [7]:
df

,EVENT_TIME,YEAR,TRACKING_ID,FACILITY,EVENT
0,2021-09-18,2021,599866340,RNO2,335
1,2021-09-16,2021,599626959,EWR2,14
2,2021-09-18,2021,601308188,IND3,335
3,2021-09-18,2021,601259821,IND3,335
4,2021-09-18,2021,599707702,IND3,335
...,...,...,...,...,...
461008,2021-12-07,2021,668750605,ONT1,335
461009,2021-12-07,2021,663837682,ONT1,14
461010,2021-12-07,2021,662546392,EWR2,335
461011,2021-12-07,2021,674117330,EWR2,7


In [8]:
df['EVENT'].value_counts()

14     36408209
335    32566515
7      21471738
Name: EVENT, dtype: int64

In [9]:
#df['EVENT'] = df['EVENT'].astype(str)
df['TRACKING_ID'] = df['TRACKING_ID'].astype(str)

In [10]:
df['EVENT_TIME'] = pd.to_datetime(df['EVENT_TIME'], dayfirst=True)

In [11]:
df2=df[df['EVENT'].isin([7,335])]


In [12]:
df2['EVENT_TIME'] = pd.to_datetime(df2['EVENT_TIME'], format='%d.%m.%Y')
#create a temporary DataFrame
df3 = df2.groupby(['EVENT_TIME', 'TRACKING_ID','EVENT']).aggregate(n=('EVENT','count')).reset_index()
#calc the lastweek
# df2['LastWeek_Count'] = (df2.groupby(['Company', 'Country'])
#                             .rolling(8, min_periods=1, on = 'Date')['Sold']
#                             .sum().reset_index(drop=True)
#                         ) 

In [ ]:
df1=df
date = pd.DatetimeIndex(df1['EVENT_TIME'])

# compute df2: totals by month
df1['month'] = date.to_period('M')
df2 = df1[df1['EVENT'].isin([7,335]) ].groupby(['TRACKING_ID', 'month']).sum()

# compute df3: totals by last seven days
from datetime import timedelta
is_last_seven = date.to_period('M') != (date + timedelta(days=7)).to_period('M')
df3 = df1[(df1['EVENT'] != 14) & is_last_seven].groupby(['TRACKING_ID', df1.month + 1]).sum()

# join the results
result = df2.join(df3, rsuffix='_last_seven')
result=result.fillna(0)

In [ ]:
delta = 7
df5=df[['EVENT_TIME','FACILITY']]
df5['count_in_last_%s_days' %(delta)] = df5.assign(count=1).groupby(
    ['EVENT_TIME']).apply(lambda x: x.rolling('%sD' %delta, on='EVENT_TIME').sum(
        ))['count'].astype(int) - 1

In [ ]:
df4 = (df.groupby(["TRACKING_ID","EVENT"])
                            .rolling(8, min_periods=1, on = 'EVENT_TIME')
                            .sum().reset_index(drop=True)
                        ) 
df4

In [ ]:
delta = 7
df4=df[['EVENT_TIME','FACILITY']]
df4['count_in_last_%s_days' %(delta)] = df4.groupby(
    ['EVENT_TIME','FACILITY']).apply(lambda x: x.rolling('%sD' %delta, on='EVENT_TIME').count()).astype(int) 

In [ ]:
df.groupby(['TRACKING_ID','FACILITY']).sum()

In [ ]:
df4 = (df3.groupby(["TRACKING_ID","EVENT"])
                            .rolling(8, min_periods=1, on = 'EVENT_TIME')
                            .sum().reset_index(drop=True)
                        ) 

In [ ]:
df4

In [ ]:
df.info()

In [ ]:
print(df.groupby(['FACILITY']).count().sort_values('EVENT', ascending=False))
print("---------------------------------")
print(df.groupby(['EVENT']).count().sort_values('FACILITY', ascending=False))
print("---------------------------------")
# print(df.groupby(['FACILITY','EVENT']).count().sort_values('ACTUAL', ascending=False))
print("---------------------------------")
df=df[(df['FACILITY'].isin(['IND3', 'EWR2', 'ATL4', 'ONT1', 'RNO2', 'DFW5', 'CVG1', 'ORD2']))]
#     print("---------------------------------")
#     df=df[(df['CATEGORY_1'].isin(['7', '33', '334', '335']))]
#   print("---------------------------------")
# print(df.groupby(['CATEGORY_1','CATEGORY_2']).count().sort_values('ACTUAL', ascending=False))
# print("---------------------------------")

In [ ]:
df1 = df[df['EVENT'].isin(['7','326']) & ~df['EVENT'].isin(['14'])]

In [ ]:
df1

In [ ]:
df2 = df1.groupby(['EVENT_TIME','FACILITY'],as_index=False)['TRACKING_ID'].nunique()

In [ ]:
print(df2)

In [ ]:
df

In [ ]:
def move_group(group):
    if not any(group['EVENT']==14):
        return group

df_ = df.groupby('TRACKING_ID').apply(move_group).dropna()
df_

In [ ]:
df4=df
df4

In [ ]:
####Start here
df5=df4.head(1000000)
df5['EVENT_TIME']=pd.to_datetime(df5['EVENT_TIME'])

In [ ]:
df5=df5[df4['EVENT'].isin([7,335,14])]
df6=df5.set_index('EVENT_TIME').\
groupby(['TRACKING_ID','FACILITY',pd.Grouper(freq='7D')]).\
filter(lambda grp: (grp['EVENT'] != 14).all())






In [ ]:
df6

In [ ]:
df6.groupby(['EVENT_TIME','FACILITY']).aggregate(count=('EVENT','count')).head(60)

In [ ]:
#df5=df5[df4['EVENT'].isin([7,335,14])]
df7=df5.set_index('EVENT_TIME').\
groupby(['TRACKING_ID','FACILITY',pd.Grouper(freq='1D')]).\
filter(lambda grp: (grp['EVENT'] != 14).all())



In [ ]:
df7.groupby(['EVENT_TIME','FACILITY']).aggregate(count=('EVENT','count')).head(60)

In [ ]:
#####End here

In [ ]:
df6.groupby(['EVENT_TIME','FACILITY']).aggregate(count=('EVENT','count')).head(20)

In [ ]:
df3 = df2.set_index('EVENT_TIME').groupby('FACILITY',as_index=False).rolling('3D').sum()
# df3 = df2.groupby('FACILITY',as_index=False).rolling('3D').sum()

In [ ]:
df3

In [ ]:
#EDA
exploratory_data_analysis(df)

In [ ]:
#preprocessing data to train
df=preprocessing(df,window)

#get list of category_1 and 2
category_1_list=list(df['CATEGORY_1'].unique())
category_2_list=list(df['CATEGORY_2'].unique())

In [ ]:
df

In [ ]:
#modeling
final_output=modeling_backtesting(category_1,category_2,confidence_interval,category_1_list,category_2_list,df,window,training_period,interval,number_of_forecast)

In [ ]:
final_output

In [ ]:
cleaned_output=output_cleaning(final_output,df)
print(cleaned_output)

In [ ]:
#format CSV file
results=formatted_csv(cleaned_output,monitor_name,lob,version)
results

In [ ]:
#results structure
results.info()

In [ ]:
# from snowflake.connector.pandas_tools import pd_writer
# import os

# def write_to_snowflake(data):
    
#     job_run_time=datetime.today()
#     print("Writing to Snowflake")
    
#     #Append Job Parameters in OutputDataFrame
#     data.insert(0,'JOB_ID',job_id)
# #     data.insert(1,'RUN_DATE',datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
# #     data.insert(2,'ANOMALY_NAME',job_name)

#     ctx = snowflake.connector.connect(
#     user= user_id,
#     password= passcode,
#     account= snowflake_account
#     )
#     #create a cursor object.
#     cs = ctx.cursor()

#     #Use Snowflake_Output_Schema
#     cs.execute("USE WAREHOUSE " + WAREHOUSE)
#     cs.execute("USE DATABASE " + snowflake_output_database)
#     cs.execute("USE SCHEMA " + snowflake_output_schema)
    
#     #Create File Format
#     Format_name = "ANOMOLY_FILE_FORMAT"
#     fileFormatQuery = 'CREATE OR REPLACE FILE FORMAT ' +Format_name+ ' TYPE = \'CSV\' NULL_IF = (\'\') EMPTY_FIELD_AS_NULL = TRUE field_delimiter = \'|\''
#     cs.execute(fileFormatQuery)
#     inputFileName = str(job_id)+'_'+str(job_name)+'_'+datetime.now().strftime('%Y%m%d_%H%M%S')+'.csv'
    
#     #Create Local Job Output Directory
#     os.system("mkdir -p job_output")
    
#     # save file to local
#     data.to_csv('job_output/'+inputFileName,index=False,header=False,sep='|')
    
#     # upload to stage
#     uploadToStageQuery = 'put file://job_output/'+inputFileName+' @%ANOMALY_BACKTESTING_TIMESERIES;'
#     cs.execute(uploadToStageQuery)
    
#     # copy to table
#     copyIntoQuery  = 'COPY INTO "'+snowflake_output_table+'" FROM @"%ANOMALY_BACKTESTING_TIMESERIES" FILE_FORMAT = "'+Format_name+'"'
#     cs.execute(copyIntoQuery)

#     # Remove Staged Files from Staged
#     removeStagedQuery = 'rm @%ANOMALY_BACKTESTING_TIMESERIES/'+str(inputFileName) + '.gz'
    
#     cs.execute(removeStagedQuery)
    
#     cs.close()
      
#     pass 

In [ ]:
# write_to_snowflake(results) 

In [ ]:
rate=alert_rate(confidence_interval,category_1_list,category_2_list,results,number_of_forecast)
rate